In [2]:
import requests
import json
import datetime
import numpy as np
import pandas as pd
import psycopg2
from setting import Setting

In [ ]:
import psycopg2.extras as extras

In [51]:
def query_yahoo_finance(stock_code, start, end):
    site = "https://query1.finance.yahoo.com/v8/finance/chart/{stock_code:s}?period1={start:d}&period2={end:d}&interval=1d&events=history".format(
                stock_code=stock_code, start= start, end=end)

    response = requests.get(site)
    data = json.loads(response.text)
    df = pd.DataFrame(data['chart']['result'][0]['indicators']['quote'][0],
                      index=pd.to_datetime(np.array(data['chart']['result'][0]['timestamp'])*1000*1000*1000))

    return df.dropna()

In [52]:
now_timestamp = int(datetime.datetime.now().timestamp())
# df = query_yahoo_finance('0050.TW', 0, now_timestamp)
# aa = query_yahoo_finance('0050.TW', 0, 1549258857)
df = query_yahoo_finance('VTI', 0, now_timestamp)

In [53]:
df[df.isna().any(axis=1)]

,volume,open,close,high,low


In [54]:
for row in df.iterrows():
    print(row[0])
    print(row[1])
    
    break

2001-06-15 13:30:00
volume    1.067400e+06
open      5.542500e+01
close     5.566500e+01
high      5.600500e+01
low       5.517500e+01
Name: 2001-06-15 13:30:00, dtype: float64


In [55]:
df['close'].isnull().values.any()

False

In [3]:
config = Setting('localhost')

In [83]:
df['date'] = df.index.strftime("%Y-%m-%d")

In [85]:
df.columns

Index(['volume', 'open', 'close', 'high', 'low', 'date', 'stock_code'], dtype='object')

In [90]:
stock_code = '0050.TW'
df['stock_code'] = stock_code
table = 'history'
cols = ",".join(list(df.columns))

In [91]:
tuples = [tuple(x) for x in df.to_numpy()]
# tuples

In [92]:
list(df.columns)

['volume', 'open', 'close', 'high', 'low', 'date', 'stock_code']

In [99]:
insert_sql = """
    INSERT INTO {table:s}({cols:s}) VALUES %s ON CONFLICT (stock_code, date) DO NOTHING
""".format(table=table, cols=cols)
insert_sql

#2053

'\n    INSERT INTO history(volume,open,close,high,low,date,stock_code) VALUES %s ON CONFLICT (stock_code, date) DO NOTHING\n'

In [97]:
query  = "INSERT INTO %s(%s) VALUES %%s" % ('table', 'cols')
query

'INSERT INTO table(cols) VALUES %s'

In [4]:
conn = config.db_conn()

In [11]:
today = datetime.datetime.now().strftime('%Y-%m-%d')
today

'2020-11-01'

In [9]:
query_sql = """
    SELECT EXISTS(
        SELECT 1
        FROM history as h
        WHERE h.stock_code = %(stock_code)s and h.date = %(date)s
    );
"""

In [24]:
# res_df = pd.read_sql(query_sql, con=conn, params={'stock_code': '0050', 'date': today})
# res_df
cursor = conn.cursor()

In [29]:
cursor.execute(query_sql, {'stock_code': '0050', 'date': today})

In [30]:
res = cursor.fetchone()
res

(False,)

In [32]:
type(res[0])

bool

In [33]:
cursor.close()

In [34]:
conn.close()

In [37]:
element = datetime.datetime.strptime(today,"%Y-%m-%d")
timestamp = int(datetime.datetime.timestamp(element))
print(timestamp)

1604160000
